# DroughtAnalysis Class Example

This notebook demonstrates how to use the DroughtAnalysis class for analyzing drought conditions, comparing across elevation bands, and performing temporal trend analysis. The DroughtAnalysis class provides an object-oriented interface to analyze drought conditions based on SSWEI values.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# Import snowdroughtindex package
from snowdroughtindex.core.dataset import SWEDataset
from snowdroughtindex.core.sswei_class import SSWEI
from snowdroughtindex.core.drought_analysis import DroughtAnalysis

## 1. Creating Sample Data for Different Elevation Bands

First, let's create sample SWE data for different elevation bands.

In [ ]:
# Function to create sample SWE data for an elevation band
def create_sample_data(elevation_band, years_range=(1980, 2021), seed=None):
    # Set random seed for reproducibility
    if seed is not None:
        np.random.seed(seed)
    
    # Create dates
    years = range(years_range[0], years_range[1] + 1)
    months = range(1, 13)  # 12 months per year
    days = [1]  # Just the first day of each month for simplicity
    
    dates = []
    for year in years:
        for month in months:
            for day in days:
                try:
                    dates.append(datetime(year, month, day))
                except ValueError:
                    # Skip invalid dates
                    pass
    
    # Create stations
    stations = [f'{elevation_band}_station_{i}' for i in range(1, 4)]
    
    # Extract elevation value from band name (e.g., "1500-2000m" -> 1750)
    try:
        elevation_str = elevation_band.replace('m', '').split('-')
        if len(elevation_str) == 2:
            min_elev = float(elevation_str[0])
            max_elev = float(elevation_str[1])
            mid_elev = (min_elev + max_elev) / 2
        else:
            mid_elev = float(elevation_band.replace('m', ''))
    except:
        mid_elev = 1000  # Default value
    
    # Create a DataFrame with SWE values
    data = {}
    for station in stations:
        # Create seasonal pattern with random noise and elevation-dependent factors
        values = []
        for i, date in enumerate(dates):
            # Seasonal pattern (higher in winter, lower in summer)
            month_factor = np.sin(2 * np.pi * (date.month - 1) / 12 - np.pi/2)
            
            # Base SWE increases with elevation
            elevation_factor = mid_elev / 1000  # Scale factor based on elevation
            
            # Higher elevations have more snow
            seasonal = elevation_factor * 100 * max(0, month_factor) + 10
            
            # Add some year-to-year variability (drought cycles)
            # Different elevation bands may have different drought patterns
            year_factor = np.sin(2 * np.pi * (date.year - 1980) / 20 + (mid_elev % 1000) / 1000 * np.pi)
            year_effect = 30 * year_factor
            
            # Add random noise
            noise = np.random.normal(0, 10)
            
            # Combine factors
            value = seasonal + year_effect + noise
            value = max(0, value)  # No negative SWE values
            values.append(value)
        
        data[station] = values
    
    df = pd.DataFrame(data, index=dates)
    
    # Create station information
    station_info = pd.DataFrame({
        'station_id': stations,
        'lat': np.random.uniform(40, 45, len(stations)),
        'lon': np.random.uniform(-120, -115, len(stations)),
        'elevation': np.random.uniform(min_elev, max_elev, len(stations)) if 'min_elev' in locals() else np.random.uniform(mid_elev - 100, mid_elev + 100, len(stations))
    })
    
    # Create SWEDataset
    return SWEDataset(df, station_info)

In [ ]:
# Create sample data for different elevation bands
elevation_bands = ['1000-1500m', '1500-2000m', '2000-2500m', '2500-3000m']
datasets = {}

for i, band in enumerate(elevation_bands):
    datasets[band] = create_sample_data(band, seed=42 + i)
    print(f"Created sample data for {band}")

## 2. Creating a DroughtAnalysis Object

Now, let's create a DroughtAnalysis object and add our datasets.

In [ ]:
# Create a DroughtAnalysis object
drought_analysis = DroughtAnalysis()

# Add datasets for each elevation band
for band, dataset in datasets.items():
    drought_analysis.add_dataset(band, dataset, is_elevation_band=True)

print(drought_analysis)

## 3. Calculating SSWEI for All Datasets

Let's calculate SSWEI for all datasets for the winter season (December to March).

In [ ]:
# Calculate SSWEI for all datasets
try:
    drought_analysis.calculate_sswei(
        start_month=12,  # December
        end_month=3,     # March
        min_years=10,    # Minimum 10 years of data required
        distribution='gamma'  # Use gamma distribution
    )
    
    print("SSWEI calculated for all datasets.")
    
    # Display SSWEI data for the first elevation band
    first_band = elevation_bands[0]
    print(f"\nSSWEI data for {first_band}:")
    display(drought_analysis.sswei_objects[first_band].sswei_data.head())
except Exception as e:
    print(f"Error calculating SSWEI: {e}")

## 4. Comparing Drought Conditions Across Elevation Bands

Let's compare drought conditions across different elevation bands.

In [ ]:
# Compare drought conditions across elevation bands
try:
    comparison_df = drought_analysis.compare_elevation_bands()
    
    print("Drought comparison across elevation bands:")
    display(comparison_df)
    
    # Plot the comparison
    fig = drought_analysis.plot_elevation_band_comparison()
    plt.show()
except Exception as e:
    print(f"Error comparing elevation bands: {e}")

## 5. Analyzing Temporal Trends in Drought Conditions

Let's analyze temporal trends in drought conditions using a moving window approach.

In [ ]:
# Analyze temporal trends in drought conditions
try:
    trend_results = drought_analysis.analyze_temporal_trends(window_size=10)
    
    # Display trend data for the first elevation band
    first_band = elevation_bands[0]
    print(f"Temporal trend data for {first_band}:")
    display(trend_results[first_band].head())
    
    # Plot temporal trends
    fig = drought_analysis.plot_temporal_trends()
    plt.show()
except Exception as e:
    print(f"Error analyzing temporal trends: {e}")

## 6. Analyzing Drought Characteristics

Let's analyze drought characteristics such as duration, severity, and intensity for each elevation band.

In [ ]:
# Analyze drought characteristics
try:
    characteristics_results = drought_analysis.analyze_drought_characteristics()
    
    # Display characteristics for each elevation band
    for band, chars_data in characteristics_results.items():
        if not chars_data.empty:
            print(f"\nDrought characteristics for {band}:")
            display(chars_data.head())
        else:
            print(f"\nNo drought events found for {band}.")
except Exception as e:
    print(f"Error analyzing drought characteristics: {e}")

## 7. Analyzing Drought Synchronicity

Let's analyze the synchronicity of drought events across elevation bands.

In [ ]:
# Analyze drought synchronicity
try:
    sync_data = drought_analysis.analyze_drought_synchronicity()
    
    print("Drought synchronicity data:")
    display(sync_data.head())
    
    # Plot drought synchronicity
    heatmap_fig, timeseries_fig = drought_analysis.plot_drought_synchronicity()
    plt.figure(heatmap_fig.number)
    plt.show()
    plt.figure(timeseries_fig.number)
    plt.show()
except Exception as e:
    print(f"Error analyzing drought synchronicity: {e}")

## 8. Analyzing Elevation Sensitivity

Let's analyze the sensitivity of drought conditions to elevation.

In [ ]:
# Analyze elevation sensitivity
try:
    elevation_df = drought_analysis.analyze_elevation_sensitivity()
    
    print("Elevation sensitivity data:")
    display(elevation_df)
    
    # Plot elevation sensitivity
    fig = drought_analysis.plot_elevation_sensitivity()
    plt.show()
except Exception as e:
    print(f"Error analyzing elevation sensitivity: {e}")

## 9. Comparing SSWEI Values for a Specific Year

Let's compare SSWEI values for a specific year across all elevation bands.

In [ ]:
# Compare SSWEI values for a specific year
try:
    # Choose a year (e.g., 2015)
    year = 2015
    
    # Plot SSWEI comparison
    fig = drought_analysis.plot_sswei_comparison(year)
    plt.show()
except Exception as e:
    print(f"Error comparing SSWEI values: {e}")

## 10. Exporting Analysis Results

Finally, let's export the analysis results to CSV files.

In [ ]:
# Export analysis results
try:
    output_dir = '../data/processed/drought_analysis_results'
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Export results
    drought_analysis.export_results(output_dir)
except Exception as e:
    print(f"Error exporting analysis results: {e}")

## 11. Summary

In this notebook, we've demonstrated how to use the DroughtAnalysis class for analyzing drought conditions, comparing across elevation bands, and performing temporal trend analysis. We've covered:

1. Creating sample data for different elevation bands
2. Creating a DroughtAnalysis object
3. Calculating SSWEI for all datasets
4. Comparing drought conditions across elevation bands
5. Analyzing temporal trends in drought conditions
6. Analyzing drought characteristics
7. Analyzing drought synchronicity
8. Analyzing elevation sensitivity
9. Comparing SSWEI values for a specific year
10. Exporting analysis results

The DroughtAnalysis class provides a convenient object-oriented interface for analyzing drought conditions based on SSWEI values, making it easier to compare drought conditions across elevation bands and perform temporal trend analysis.